In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
trips = pd.read_csv("trips.csv")
trips

,activityid,personid,modechoice
0,1,3,WALK
1,2,3,WALK
2,3,3,WALK
3,4,3,WALK
4,5,3,WALK
...,...,...,...
14047024,17147588,4113994,DRIVE_ALONE_FREE
14047025,17147589,4113994,DRIVE_ALONE_FREE
14047026,17147590,4113994,DRIVE_ALONE_FREE
14047027,17147592,4113995,SHARED_3_HOV


In [4]:
trips.isnull().sum()

activityid    0
personid      0
modechoice    0
dtype: int64

In [5]:
trips['modechoice'].value_counts()

DRIVE_ALONE_FREE    6156506
SHARED_2_HOV        3507285
SHARED_3_HOV        2814408
WALK                1022834
WALK_SET             214366
SCH_BUS              194123
BIKE                  69645
SHARED_3_PAY          17088
SHARED_2_PAY          15450
KNR_SET               13633
PNR_SET               11926
DRIVE_ALONE_PAY        9765
Name: modechoice, dtype: int64

In [6]:
utilityvars = pd.read_csv("utilityvars.csv")
utilityvars

,activityid,age,gender,autosuf,numhouseholdpersons,income,oduden,oempden,ototint,dempden,sovdrivetime,hovdrivetime,tolldrivetime,tollcostsov,tollcosthov2,tollcosthov3,walkttime,walktotransitutility,drivetotransitutility,parkingcost,parkingwalktime,sovcost,hovcost,tollcost,tourpurpose,tourmode,firststop,laststop,zerototalstops,targettripmode
0,12150818,4,True,2,2,4,9.633200,11.594058,4.0,67.659134,5.053318,5.053318,5.053318,0.0,0.0,0.0,10.677524,-999.0,-999.0,0.0,0.0,46.447229,46.447229,46.447229,0,5,True,False,False,1
1,3018709,6,False,2,2,1,8.304700,70.117912,12.0,32.273338,31.226185,31.226185,31.226185,0.0,0.0,0.0,113.844883,-999.0,-999.0,0.0,0.0,495.225231,495.225231,495.225231,0,1,False,False,False,1
2,8809200,6,False,2,6,3,10.189500,47.538746,3.0,13.567157,30.145569,30.145569,30.145569,0.0,0.0,0.0,106.238420,-999.0,-999.0,0.0,0.0,462.137117,462.137117,462.137117,0,1,False,False,True,1
3,11332534,4,False,2,1,2,13.393200,21.983759,4.0,51.544109,24.318232,24.318232,24.318232,0.0,0.0,0.0,78.687684,-999.0,-999.0,0.0,0.0,342.291418,342.291418,342.291418,0,1,False,False,False,1
4,6130767,6,False,2,3,2,13.274000,6.568953,3.0,8.480003,25.267041,25.267041,25.267041,0.0,0.0,0.0,71.307144,-999.0,-999.0,0.0,0.0,310.186071,310.186071,310.186071,0,1,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14047024,10138525,2,True,2,4,2,35.460899,286.328949,14.0,12.385718,56.546418,56.546418,56.546418,0.0,0.0,0.0,193.717741,-999.0,-999.0,0.0,0.0,842.672156,842.672156,842.672156,5,5,False,False,False,5
14047025,16270294,7,False,2,4,3,17.207199,19.868286,13.0,1.423494,1.403090,1.403090,1.403090,0.0,0.0,0.0,3.069109,-999.0,-999.0,0.0,0.0,13.350623,13.350623,13.350623,5,3,False,False,False,3
14047026,16270295,7,False,2,4,3,16.574200,1.423494,13.0,88.151642,12.287907,12.287907,12.287907,0.0,0.0,0.0,31.580930,-999.0,-999.0,0.0,0.0,137.377041,137.377041,137.377041,5,3,False,False,False,3
14047027,10138526,2,True,2,4,2,22.916100,12.385718,14.0,8.472905,1.173692,1.173692,1.173692,0.0,0.0,0.0,1.252347,-999.0,-999.0,0.0,0.0,5.447711,5.447711,5.447711,5,5,False,False,False,3


In [7]:
# is there any missingness?
utilityvars.isnull().sum()

activityid               0
age                      0
gender                   0
autosuf                  0
numhouseholdpersons      0
income                   0
oduden                   0
oempden                  0
ototint                  0
dempden                  0
sovdrivetime             0
hovdrivetime             0
tolldrivetime            0
tollcostsov              0
tollcosthov2             0
tollcosthov3             0
walkttime                0
walktotransitutility     0
drivetotransitutility    0
parkingcost              0
parkingwalktime          0
sovcost                  0
hovcost                  0
tollcost                 0
tourpurpose              0
tourmode                 0
firststop                0
laststop                 0
zerototalstops           0
targettripmode           0
dtype: int64

In [8]:
utilityvars['parkingcost'].value_counts()

0.000000      13216142
125.000000       95773
943.000000       81421
412.000000       37549
137.604996       21844
                ...   
381.760692           1
89.240509            1
152.255060           1
2.973118             1
285.949867           1
Name: parkingcost, Length: 59411, dtype: int64

In [9]:
# sample the whole dataset

mode1 = utilityvars[utilityvars['targettripmode']==1].sample(n=1000, weights='walkttime', random_state=1)
mode2 = utilityvars[utilityvars['targettripmode']==2].sample(n=1000, weights='walkttime', random_state=1)
mode3 = utilityvars[utilityvars['targettripmode']==3].sample(n=1000, weights='walkttime', random_state=1)
mode4 = utilityvars[utilityvars['targettripmode']==4].sample(n=1000, weights='walkttime', random_state=1)
mode5 = utilityvars[utilityvars['targettripmode']==5].sample(n=1000, weights='walkttime', random_state=1)
mode6 = utilityvars[utilityvars['targettripmode']==6].sample(n=1000, weights='walkttime', random_state=1)
mode7 = utilityvars[utilityvars['targettripmode']==7].sample(n=1000, weights='walkttime', random_state=1)
mode8 = utilityvars[utilityvars['targettripmode']==8].sample(n=1000, weights='walkttime', random_state=1)
mode9 = utilityvars[utilityvars['targettripmode']==9].sample(n=1000, weights='walkttime', random_state=1)
mode10 = utilityvars[utilityvars['targettripmode']==10].sample(n=1000, weights='walkttime', random_state=1)
mode11 = utilityvars[utilityvars['targettripmode']==11].sample(n=1000, weights='walkttime', random_state=1)
mode12 = utilityvars[utilityvars['targettripmode']==12].sample(n=1000, weights='walkttime', random_state=1)

In [10]:
combined_subsamples = pd.concat([mode1,mode2,mode3,mode4,mode5,mode6,mode7,mode8,mode9,mode10,mode11,mode12], axis=0)

In [11]:
# One-hot encode the categorical variables (the variables only include 0 and -1 don't need to be encoded)
new_utilityvars = pd.get_dummies(combined_subsamples, columns=["tourpurpose", "tourmode"], prefix=["tourpurpose", "tourmode"])
new_utilityvars

,activityid,age,gender,autosuf,numhouseholdpersons,income,oduden,oempden,ototint,dempden,sovdrivetime,hovdrivetime,tolldrivetime,tollcostsov,tollcosthov2,tollcosthov3,walkttime,walktotransitutility,drivetotransitutility,parkingcost,parkingwalktime,sovcost,hovcost,tollcost,firststop,laststop,zerototalstops,targettripmode,tourpurpose_0,tourpurpose_1,tourpurpose_2,tourpurpose_3,tourpurpose_4,tourpurpose_5,tourpurpose_6,tourpurpose_7,tourmode_1,tourmode_3,tourmode_5,tourmode_7,tourmode_8,tourmode_9,tourmode_10,tourmode_11,tourmode_12
5420299,5933728,7,True,2,2,2,15.0496,6.528248,3.0,28.221581,25.183311,25.183311,25.183311,0.0,0.0,0.0,74.541791,-999.0,-999.0,0.0,0.0,324.256782,324.256782,324.256782,False,False,True,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
10137151,14562853,3,False,2,5,4,3.4107,230.852676,22.0,27.689655,20.561739,20.561739,20.561739,0.0,0.0,0.0,68.671031,-999.0,-999.0,0.0,0.0,298.718978,298.718978,298.718978,False,False,True,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
459,13897599,6,False,1,5,5,7.3797,4.521993,1.0,137.408112,32.240913,32.231399,32.240913,0.0,0.0,0.0,125.903913,-999.0,-999.0,0.0,0.0,547.682008,547.825087,547.682008,False,False,False,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3523186,5757716,7,False,1,2,4,6.2555,31.995995,1.0,6.055996,9.218620,9.218620,9.218620,0.0,0.0,0.0,30.619256,-999.0,-999.0,0.0,0.0,133.193759,133.193759,133.193759,False,False,False,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1389833,9001967,7,False,2,1,2,13.1061,80.675232,3.0,24.416708,14.847696,14.847696,14.847696,0.0,0.0,0.0,51.645009,-999.0,-999.0,0.0,0.0,224.655784,224.655784,224.655784,False,False,False,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4649022,5565944,1,False,2,5,5,3.6818,46.511749,1.0,90.145737,5.535691,5.535691,5.535691,0.0,0.0,0.0,12.468397,-999.0,-999.0,0.0,0.0,54.237525,54.237525,54.237525,False,False,True,12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1014748,11693791,1,True,1,4,4,7.7022,19.270676,12.0,42.764721,2.246100,2.246100,2.246100,0.0,0.0,0.0,5.529111,-999.0,-999.0,0.0,0.0,24.051633,24.051633,24.051633,False,False,True,12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
409363,512651,3,False,2,5,1,5.6995,14.991196,10.0,46.152000,19.167122,19.167122,19.167122,0.0,0.0,0.0,69.843901,-999.0,-999.0,0.0,0.0,303.820963,303.820963,303.820963,False,False,True,12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1561740,101043,1,True,1,3,2,6.4687,20.669909,8.0,19.647785,15.057140,15.057140,15.057140,0.0,0.0,0.0,34.377707,-999.0,-999.0,0.0,0.0,149.543022,149.543022,149.543022,False,False,True,12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [12]:
# Build XGBoost tree model
new_utilityvars = pd.merge(new_utilityvars.drop(['targettripmode'], axis=1), new_utilityvars[['targettripmode']], left_index=True, right_index=True, how="outer")
new_utilityvars

,activityid,age,gender,autosuf,numhouseholdpersons,income,oduden,oempden,ototint,dempden,sovdrivetime,hovdrivetime,tolldrivetime,tollcostsov,tollcosthov2,tollcosthov3,walkttime,walktotransitutility,drivetotransitutility,parkingcost,parkingwalktime,sovcost,hovcost,tollcost,firststop,laststop,zerototalstops,tourpurpose_0,tourpurpose_1,tourpurpose_2,tourpurpose_3,tourpurpose_4,tourpurpose_5,tourpurpose_6,tourpurpose_7,tourmode_1,tourmode_3,tourmode_5,tourmode_7,tourmode_8,tourmode_9,tourmode_10,tourmode_11,tourmode_12,targettripmode
5420299,5933728,7,True,2,2,2,15.0496,6.528248,3.0,28.221581,25.183311,25.183311,25.183311,0.0,0.0,0.0,74.541791,-999.0,-999.0,0.0,0.0,324.256782,324.256782,324.256782,False,False,True,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1
10137151,14562853,3,False,2,5,4,3.4107,230.852676,22.0,27.689655,20.561739,20.561739,20.561739,0.0,0.0,0.0,68.671031,-999.0,-999.0,0.0,0.0,298.718978,298.718978,298.718978,False,False,True,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
459,13897599,6,False,1,5,5,7.3797,4.521993,1.0,137.408112,32.240913,32.231399,32.240913,0.0,0.0,0.0,125.903913,-999.0,-999.0,0.0,0.0,547.682008,547.825087,547.682008,False,False,False,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3523186,5757716,7,False,1,2,4,6.2555,31.995995,1.0,6.055996,9.218620,9.218620,9.218620,0.0,0.0,0.0,30.619256,-999.0,-999.0,0.0,0.0,133.193759,133.193759,133.193759,False,False,False,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1389833,9001967,7,False,2,1,2,13.1061,80.675232,3.0,24.416708,14.847696,14.847696,14.847696,0.0,0.0,0.0,51.645009,-999.0,-999.0,0.0,0.0,224.655784,224.655784,224.655784,False,False,False,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4649022,5565944,1,False,2,5,5,3.6818,46.511749,1.0,90.145737,5.535691,5.535691,5.535691,0.0,0.0,0.0,12.468397,-999.0,-999.0,0.0,0.0,54.237525,54.237525,54.237525,False,False,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12
1014748,11693791,1,True,1,4,4,7.7022,19.270676,12.0,42.764721,2.246100,2.246100,2.246100,0.0,0.0,0.0,5.529111,-999.0,-999.0,0.0,0.0,24.051633,24.051633,24.051633,False,False,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12
409363,512651,3,False,2,5,1,5.6995,14.991196,10.0,46.152000,19.167122,19.167122,19.167122,0.0,0.0,0.0,69.843901,-999.0,-999.0,0.0,0.0,303.820963,303.820963,303.820963,False,False,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12
1561740,101043,1,True,1,3,2,6.4687,20.669909,8.0,19.647785,15.057140,15.057140,15.057140,0.0,0.0,0.0,34.377707,-999.0,-999.0,0.0,0.0,149.543022,149.543022,149.543022,False,False,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12


In [13]:
df_array = new_utilityvars.values
X = df_array[:,0:44]
Y = df_array[:,44]

In [14]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [15]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 0)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:01:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [16]:
y_pred = [round(value) for value in model.predict(X_test)]
y_test = [round(value) for value in y_test]

In [17]:
# multilabel confusion matrix
metrics.multilabel_confusion_matrix(y_test, y_pred)

array([[[3604,   26],
        [  65,  265]],

       [[3617,   29],
        [  41,  273]],

       [[3535,   85],
        [  86,  254]],

       [[3580,   43],
        [  80,  257]],

       [[3507,  113],
        [  45,  295]],

       [[3558,   64],
        [  42,  296]],

       [[3596,   32],
        [  61,  271]],

       [[3645,    0],
        [   0,  315]],

       [[3586,   33],
        [  14,  327]],

       [[3634,    2],
        [   0,  324]],

       [[3632,   10],
        [   3,  315]],

       [[3629,    0],
        [   0,  331]]])

In [18]:
# accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.8896464646464647

In [19]:
# sensitivity
sensitivity = metrics.recall_score(y_test, y_pred, average = 'macro')
sensitivity

0.8909407697843456

In [20]:
# precision
precision = metrics.precision_score(y_test, y_pred, average = 'macro')
precision

0.8943029703772615

In [21]:
# F1-score
f1 = (2 * precision * sensitivity) / (precision + sensitivity)
f1

0.8926187040165365

In [22]:
# 13*13 confusion matrix
metrics.confusion_matrix(y_test, y_pred)

array([[265,   5,  34,   0,  20,   1,   0,   0,   4,   1,   0,   0],
       [  3, 273,   0,  17,   2,  19,   0,   0,   0,   0,   0,   0],
       [ 10,   0, 254,   5,  45,   1,  16,   0,   6,   0,   3,   0],
       [  1,  22,  15, 257,   2,  38,   1,   0,   0,   0,   1,   0],
       [  6,   0,  16,   1, 295,   4,  10,   0,   5,   0,   3,   0],
       [  0,   2,   1,  18,  19, 296,   1,   0,   1,   0,   0,   0],
       [  6,   0,  16,   0,  21,   0, 271,   0,  17,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0,   0, 315,   0,   0,   0,   0],
       [  0,   0,   2,   0,   4,   1,   4,   0, 327,   1,   2,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 324,   0,   0],
       [  0,   0,   1,   2,   0,   0,   0,   0,   0,   0, 315,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 331]])

In [26]:
# normalized confusion matrix

|  | pred Drive Alone Free | pred Drive Alone Pay | pred HOV2 Free | pred HOV2 Pay | pred HOV3 Free | pred HOV3 Pay | pred Walk | pred Bike | pred Walk to Transit | pred Park and Ride | pred Kiss and Ride | pred School Bus |
| :- | :- | :- | :- | :- | :- | :- | :- | :- | :- | :- | :- | :- |
| Drive Alone Free | 0.8030303 | 0.01515152 | 0.1030303 | 0 | 0.06060606 | 0.0030303 | 0 | 0 | 0.01212121 | 0.0030303 | 0 | 0 |
| Drive Alone Pay | 0.00955414 | 0.86942675 | 0 | 0.05414013 | 0.00636943 | 0.06050955 | 0 | 0 | 0 | 0 | 0 | 0 |
| HOV2 Free | 0.02941176 | 0 | 0.74705882 | 0.01470588 | 0.13235294 | 0.00294118 | 0.04705882 | 0 | 0.01764706 | 0 | 0.00882353 | 0 |
| HOV2 Pay | 0.00296736 | 0.0652819 | 0.04451039 | 0.76261128 | 0.00593472 | 0.11275964 | 0.00296736 | 0 | 0 | 0 | 0.00296736 | 0 |
| HOV3 Free | 0.01764706 | 0 | 0.04705882 | 0.00294118 | 0.86764706 | 0.01176471 | 0.02941176 | 0 | 0.01470588 | 0 | 0.00882353 | 0 |
| HOV3 Pay | 0 | 0.00591716 | 0.00295858 | 0.05325444 | 0.05621302 | 0.87573964 | 0.00295858 | 0 | 0.00295858 | 0 | 0 | 0 |
| Walk | 0.01807229 | 0 | 0.04819277 | 0 | 0.06325301 | 0 | 0.81626506 | 0 | 0.05120482 | 0 | 0.00301205 | 0 |
| Bike | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| Walk to Transit | 0 | 0 | 0.0058651 | 0 | 0.01173021 | 0.00293255 | 0.01173021 | 0 | 0.95894428 | 0.00293255 | 0.0058651 | 0 |
| Park and Ride | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 |
| Kiss and Ride | 0 | 0 | 0.00314465 | 0.00628931 | 0 | 0 | 0 | 0 | 0 | 0 | 0.99056604 | 0 |
| School Bus | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |

In [50]:
# metrics.confusion_matrix(y_test, y_pred)[11] / sum(metrics.confusion_matrix(y_test, y_pred)[11])